# Data Wrangling and Cleaning

In [1]:
import pandas as pd

In [2]:
df_derechos_laborales = pd.read_csv(r'migracion_charly\regulacion_juridica_al_empleo.csv', sep=',', header=0)

Elimino columnas irrelevantes para el proyecto y/o redundantes

In [3]:
df_derechos_laborales.drop('source', axis=1, inplace=True)
df_derechos_laborales.drop('source.label', axis=1, inplace=True)
df_derechos_laborales.drop('indicator', axis=1, inplace=True)
df_derechos_laborales.drop('indicator.label',axis=1, inplace=True)

Cambio el nombre de las columnas

In [4]:
df_derechos_laborales.rename(columns={'ref_area':'codigo_pais', 'ref_area.label':'pais','time':'anio','obs_value':'derechos_laborales'}, inplace=True)

In [5]:
remover = ['X01', 'X02', 'X03', 'X04', 'X05', 'X06', 'X07', 'X08', 'X09', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 
        'X21', 'X23', 'X24', 'X25', 'X26', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X38', 'X42', 'X44', 'X48', 'X49', 'X51', 
        'X52', 'X53', 'X54', 'X55', 'X56', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 
        'X73', 'X74', 'X75', 'X76', 'X77', 'X78', 'X79', 'X82', 'X83', 'X84', 'X86', 'X87', 'X89', 'X90', 'X91', 'X92', 'X94']

In [6]:
indices = df_derechos_laborales[df_derechos_laborales["codigo_pais"].isin(remover)].index

In [7]:
df_derechos_laborales.drop(indices, inplace=True)

In [8]:
df_derechos_laborales.reset_index(drop=True, inplace=True)

In [51]:
df_derechos_laborales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   codigo_pais         775 non-null    object 
 1   pais                775 non-null    object 
 2   anio                775 non-null    int64  
 3   derechos_laborales  775 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 24.3+ KB


In [10]:
df_human_rights = pd.read_csv(r'migracion_charly\distribution-human-rights.csv', sep=',', header=0)

In [11]:
df_human_rights.rename(columns={'Entity':'pais', 'Code':'codigo_pais', 'Year':'anio', 'civ_libs_vdem_owid': 'derechos_humanos'}, inplace=True)

In [12]:
df_human_rights.dropna(subset=['derechos_humanos'], inplace=True)

In [13]:
df_human_rights.dropna(subset=['codigo_pais'], inplace=True)

In [14]:
df_human_rights = df_human_rights[(df_human_rights['anio'] >= 1990)]

In [15]:
indices2 = df_human_rights[(df_human_rights['codigo_pais'] == 'OWID_GDR') | (df_human_rights['codigo_pais'] == 'OWID_WRL')].index

In [16]:
df_human_rights.drop(indices2, inplace=True)

In [17]:
df_human_rights.drop('region', axis=1, inplace=True)

In [52]:
df_human_rights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5647 entries, 201 to 33625
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   pais              5647 non-null   object 
 1   codigo_pais       5647 non-null   object 
 2   anio              5647 non-null   int64  
 3   derechos_humanos  5647 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 220.6+ KB


In [19]:
df_gender_wage_gap = pd.read_csv(r'migracion_charly\gender_wage_gap.csv', sep=',', header=0)

In [20]:
df_gender_wage_gap.drop('INDICATOR', axis=1, inplace=True)
df_gender_wage_gap.drop('MEASURE', axis=1, inplace=True)
df_gender_wage_gap.drop('FREQUENCY', axis=1, inplace=True)
df_gender_wage_gap.drop('Flag Codes', axis=1, inplace=True)

In [21]:
df_gender_wage_gap.rename(columns={'LOCATION':'codigo_pais', 'SUBJECT':'tipo', 'TIME':'anio', 'Value':'valor'}, inplace=True)

In [22]:
df_gender_wage_gap = df_gender_wage_gap[(df_gender_wage_gap['anio'] >= 1990)]

In [23]:
indices3 = df_gender_wage_gap[(df_gender_wage_gap['codigo_pais'] == 'EU27') | (df_gender_wage_gap['codigo_pais'] == 'OECD')].index

In [24]:
df_gender_wage_gap.drop(indices3, inplace=True)

In [25]:
df_gender_wage_gap_employee = df_gender_wage_gap[df_gender_wage_gap['tipo'] == 'EMPLOYEE'].rename(columns={'valor':'wage_gap_employee'}).drop(['tipo'], axis=1)

In [26]:
df_gender_wage_gap_selfemployed = df_gender_wage_gap[df_gender_wage_gap['tipo'] == 'SELFEMPLOYED'].rename(columns={'valor':'wage_gap_selfemployed'}).drop(['tipo'], axis=1)

In [27]:
df_gender_wage_gap_merged = pd.merge(df_gender_wage_gap_employee, df_gender_wage_gap_selfemployed, on=['codigo_pais','anio'], how='outer')

In [28]:
df_gini = pd.read_csv(r'Etl\gini.csv')

In [29]:
df_gini.rename(columns={'Code':'codigo_pais', 'Entity':'pais', 'Year':'anio'}, inplace=True)

In [30]:
df_gini = df_gini[(df_gini['anio'] >= 1990)]

In [31]:
df_gini.loc[(df_gini['pais'] == 'Argentina - urban'),'codigo_pais'] = 'ARG'
df_gini.loc[(df_gini['pais'] == 'Bolivia - urban'),'codigo_pais'] = 'BOL'
df_gini.loc[(df_gini['pais'] == 'Uruguay - urban'),'codigo_pais'] = 'URY'
df_gini.loc[(df_gini['pais'] == 'Micronesia (country) - urban'),'codigo_pais'] = 'FSM'
df_gini.loc[(df_gini['pais'] == 'Suriname - urban'),'codigo_pais'] = 'SUR'
df_gini.loc[(df_gini['pais'] == 'Colombia - urban'),'codigo_pais'] = 'COL'
df_gini.loc[(df_gini['pais'] == 'Ecuador - urban'),'codigo_pais'] = 'ECU'

In [32]:
indices4 = df_gini[(df_gini['pais'] == 'China - rural') | (df_gini['pais'] == 'Indonesia - urban') |
                   (df_gini['pais'] == 'China - urban') | (df_gini['pais'] == 'Indonesia - rural') |
                   (df_gini['pais'] == 'India - urban') | (df_gini['pais'] == 'India - rural')].index

In [33]:
df_gini.drop(indices4, inplace=True)

In [34]:
df_gini.head()

,pais,codigo_pais,anio,Gini coefficient
0,Albania,ALB,1996,0.270103
1,Albania,ALB,2002,0.317390
2,Albania,ALB,2005,0.305957
3,Albania,ALB,2008,0.299847
4,Albania,ALB,2012,0.289605


In [35]:
merge1 = pd.merge(df_derechos_laborales, df_human_rights, on=['codigo_pais','anio'], how='outer')

In [36]:
merge1['pais_x'].fillna(merge1['pais_y'], inplace=True)

In [37]:
merge1.drop('pais_y', axis=1, inplace=True)

In [38]:
merge2 = pd.merge(merge1, df_gini, on=['codigo_pais','anio'], how='outer')

In [39]:
merge2['pais_x'].fillna(merge2['pais'], inplace=True)

In [40]:
merge2.drop('pais', axis=1, inplace=True)

In [41]:
fact_ind_sociopolitico = pd.merge(merge2, df_gender_wage_gap_merged, on=['codigo_pais',	'anio'], how='outer')

In [42]:
cuenta = fact_ind_sociopolitico['codigo_pais'].value_counts()

In [43]:
codigos = cuenta[cuenta<10].index

In [44]:
indices5 = fact_ind_sociopolitico[fact_ind_sociopolitico['codigo_pais'].isin(codigos)].index

In [45]:
fact_ind_sociopolitico.drop(indices5, inplace=True)

In [46]:
fact_ind_sociopolitico.sort_values(by=['codigo_pais', 'anio'], inplace=True)

In [47]:
fact_ind_sociopolitico.reset_index(drop=True, inplace=True)

In [48]:
fact_ind_sociopolitico.to_csv('fact_ind_sociopolitico.csv', index=False)

In [49]:
fact_ind_sociopolitico

,codigo_pais,pais_x,anio,derechos_laborales,derechos_humanos,Gini coefficient,wage_gap_employee,wage_gap_selfemployed
0,AFG,Afghanistan,1990,NaN,0.077,NaN,NaN,NaN
1,AFG,Afghanistan,1991,NaN,0.069,NaN,NaN,NaN
2,AFG,Afghanistan,1992,NaN,0.071,NaN,NaN,NaN
3,AFG,Afghanistan,1993,NaN,0.077,NaN,NaN,NaN
4,AFG,Afghanistan,1994,NaN,0.078,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5654,ZWE,Zimbabwe,2017,4.96,0.434,0.443371,NaN,NaN
5655,ZWE,Zimbabwe,2018,5.92,0.428,NaN,NaN,NaN
5656,ZWE,Zimbabwe,2019,4.42,0.391,0.502565,NaN,NaN
5657,ZWE,Zimbabwe,2020,5.45,0.399,NaN,NaN,NaN
